In [ ]:
import numpy
import matplotlib.pyplot as plt
from nose.tools import raises, assert_almost_equal

In [ ]:
pt = numpy.zeros((3, 2))
pt[1] = (0, 1)
pt[2] = (numpy.sqrt(1 - 0.5**2), 0.5)

plt.scatter(pt[:, 0], pt[:, 1])
plt.show()

In [ ]:
n = 1000
c = numpy.random.random((3, n))
c = c / c.sum(axis = 0)
c = numpy.array([c, c]).T

x = (pt*c).sum(axis = 1)
plt.scatter(x[:, 0], x[:, 1])
plt.show()

In [ ]:
n = 10000
discard = 5
n = n + discard

xv = numpy.floor((numpy.random.random((n, 2))*3))
x0 = numpy.random.random(3); x0 = x0 / x0.sum()
xv[0, :] = ((numpy.array([x0, x0]).T)*pt).sum(axis = 0)

for idx in range(1, n):
    xv[idx, :] = (xv[idx - 1, :] + pt[int(xv[idx, 0])]) / 2

plt.scatter(xv[discard:, 0], xv[discard:, 1], s = 0.1)
plt.show()

In [ ]:
def colour(n, colours = ["red", "blue", "green"], discard = 5):
    xv = numpy.zeros((n, 2))
    x0 = numpy.random.random(3)
    x0 = x0/x0.sum()
    xv[0, :] = ((numpy.array([x0, x0]).T)*pt).sum(axis = 0)
    colour = numpy.random.randint(3, size = n)

    for  idx in range(1, n):
        xv[idx, :] = (xv[idx - 1, :] + pt[int(colour[idx])]) / 2

    red = xv[colour == 0]
    blue = xv[colour == 1]
    green = xv[colour == 2]

    plt.scatter(red[discard:, 0], red[discard:, 1], s = 0.2, c = colours[0])
    plt.scatter(blue[discard:, 0], blue[discard:, 1], s = 0.2, c = colours[1])
    plt.scatter(green[discard:, 0], green[discard:, 1], s = 0.2, c = colours[2])
    plt.show()

colour(n)

In [ ]:
def m_colour(n, discard = 5):
    rgb = numpy.zeros((n, 3))
    red = numpy.array([1, 0, 0])
    blue = numpy.array([0, 1, 0])
    green = numpy.array([0, 0, 1])
    b_rgb = numpy.array([red, blue, green])

    x0 = numpy.random.random(3)
    x0 = x0 / x0.sum()
    xv[0, :] = ((numpy.array([x0, x0]).T)*pt).sum(axis = 0)
    colour = numpy.random.randint(3, size = n)

    for idx in range(1, n): 
        xv[idx, :] = (xv[idx - 1, :] + pt[int(colour[idx])]) / 2
        rgb[idx, :] = (rgb[idx - 1, :] + b_rgb[int(colour[idx])]) / 2

    plt.scatter(xv[discard:, 0], xv[discard:, 1], s = 0.1, c = rgb[discard:])
    plt.savefig("output/rgb_triangle.png")
    plt.show()
    
m_colour(n)

In [ ]:
class ChaosGame:
    """
    Class; playing Chaos Game

    Parameters:
        n: The number of corners in n-gon
        r: Ratio between the corners per move
    """
    def __init__(self, n, r):
        if not isinstance(n, int):
            raise TypeError("Only accepts n-arguments as dtype int")
        if not isinstance(r, float):
            raise TypeError("Only accepts r-arguments as dtype float")
        if n < 3:
            raise ValueError("Only accepts n-arguments as n >= 3")
        if r <= 0 or r >= 1:
            raise ValueError("Only accepts r-arguments as 0 < r < 1")

        self.r, self.n = r, n

        self._generate_ngon()
        self._starting_point()

    def _generate_ngon(self):
        """
        Method; generating n-gon
        Parameters:
            None
        Returns:
            None
        """
        theta = numpy.linspace(0, 2*numpy.pi, self.n, endpoint=False)
        self.corners = numpy.array((numpy.sin(theta), numpy.cos(theta))).T

    def _starting_point(self):
        """
        Method; generate start-point in n-gon
        Parameters:
            None
        Returns:
            None
        """
        str_point = numpy.random.random(self.n)
        str_point = (str_point/str_point.sum())
        self.x0 = ((numpy.array([str_point, str_point]).T)*self.corners).sum(axis = 0)

    def iterate(self, steps, discard = 5):
        """
        Method; iterating in the game

        Parameters:
            steps: amount of steps the game takes.
            discard: amount of discarded points from start
        Returns:
            None
        """
        x = numpy.zeros((steps + discard, 2))

        x[0, :] = self.x0
        m  = numpy.random.randint(self.n, size=steps + discard)

        for idx in range(1, steps + discard):
            x[idx, :] = self.r*x[idx - 1, :] + (1 - self.r)*self.corners[m[idx], :]

        self.x = x[discard:,:]
        self.colour = m[discard:]

    def plot_ngon(self, make_colour):
        """
        Method; plot game that has been played

        Parameters:
            make_colour: determine black or white
        Returns:
            None
        """
        plt.xlim(-1.1, 1.1)
        plt.ylim(-1.1, 1.1)
        if make_colour:
            colours = ["red", "blue", "green", "purple", "yellow", "blue"]
            for idx in range(self.n):
                plt.scatter(self.x[:, 0][self.colour == idx],
                self.x[:, 1][self.colour == idx],
                s = 0.1, c = colours[idx % len(colours)])
        else: plt.scatter(self.x[:, 0], self.x[:, 1], c = "black", s = 0.1)

    def show(self, make_colour = True):
        """
        Method; showing plot_ngon of game

        Parameters:
            (Optional) make_colour: determine black or white
        Returns:
            None
        """
        self.plot_ngon(make_colour)
        plt.show()

    def savepng(self, filename, make_colour = True):
        """
        Method; saving plot

        Parameters:
            filename: filename for saved picture
            (Optional) make_colour: determine black or white
        Returns:
            None
        """
        if filename[-4:] == ".png":
            pass
        elif "." in filename:
            raise TypeError("Only accepts .png filetype")
        else:
            filename = filename + ".png"

        self.plot_ngon(make_colour)
        plt.savefig(filename, dpi = 300)
        plt.close()

In [ ]:
N = [3, 4, 5, 5, 6]
R = [1/2, 1/3, 1/3, 3/8, 1/3]
number = range(1, 6)

for n, r, nu in zip(N, R, number):
    c = ChaosGame(n, r)
    c.iterate(100000)
    c.savepng("output/chaos_game%i" % nu)

c = ChaosGame(10, 1/7)
c.iterate(50000)
c.savepng("output/chain", make_colour = False)
c.show(make_colour = True)

In [ ]:
def test_init():
    @raises(TypeError)
    def test_nt():
        ChaosGame(3.5, 1/3)

    @raises(TypeError)
    def test_rt():
        ChaosGame(6, "Float???")

    @raises(ValueError)
    def test_nv():
        ChaosGame(1, 1/3)

    @raises(ValueError)
    def test_rv():
        ChaosGame(5, -1/3)

    test_nt()
    test_rt()
    test_nv()
    test_rv()

def test_generate_ngon():
    n = 4; r = 1/2
    c = ChaosGame(n, r)
    assert(len(c.corners) == n)
    assert_almost_equal(c.corners.all(), numpy.array([[0, 1],[1, 0],[0, -1],[-1, 0]]).all())

@raises(TypeError)
def test_savepng():
    c = ChaosGame(8, 1/6)
    c.savepng("a_picture.raw")

def test_s_point():
    c = ChaosGame(8, 1/6)
    for idx in range(1000):
        c._starting_point()
        assert(abs(c.x0[1]) < abs(1 - c.x0[0]))

test_savepng()
test_generate_ngon()
test_init()
test_s_point()

In [ ]:
class AffineTransform:
    def __init__(self, a = 0, b = 0, c = 0, d = 0, e = 0, f = 0):
        """
        Init function.
        Parameters:
            a, b, c, d, e, f: (optional) Arrays of initial values
            for AffineTransformation.
        Returns:
            None
        """
        self.a, self.b, self.c = a, b, c
        self.d, self.e, self.f = d, e, f

    def functionTransform(self, x):
        """
        Transoform function
        Parameters:
            x: float value.
        Returns:
            [xv, yv]: numpy arrays of x- and y-values.
        """
        a, b, c = self.a, self.b, self.c
        d, e, f = self.d, self.e, self.f

        xv = x[0]*a + x[1]*b + e
        yv = x[1]*c + x[1]*d + f

        return numpy.array([xv, yv])

In [ ]:
a = [0, 0.85, 0.20,-0.15]; b = [0, 0.04,-0.26, 0.28]
c = [0, -0.04, 0.23, 0.26]; d = [0.16, 0.85, 0.22, 0.24]
e = [0, 0, 0, 0]; f = [0, 1.6, 1.6, 0.44]

p = [0.01, 0.85, 0.07, 0.07]

assert(sum(p) == 1)

f = numpy.array([AffineTransform(a[idx],b[idx],c[idx],d[idx],e[idx],f[idx])
for idx in range(len(a))])

p_cumulative = [sum(p[:idx]) for idx in range(1, len(p) + 1)]

def p_gen_func():
    r = numpy.random.random()
    for k, p in zip(range(len(p_cumulative)), p_cumulative):
        if r <= p:
            return k

n = int(50000)
discard = 5

x = numpy.zeros((n + discard, 2))
    
for idx in range(1, n + discard):
    j = idx - 1
    x[idx,:] = f[p_gen_func()].functionTransform(x[idx - 1, :])

x = x[discard:]

plt.scatter(x[:, 0], x[:, 1], s = 0.1, color = "green")
plt.xlim(-4, 4); plt.ylim(-0.5, 9.0)
plt.savefig("output/barnslay_fern.png")
plt.show()